# Stress Test性能测试
- 千帆 Python SDK 提供了基于locust工具的对大模型服务进行快速压测以及性能评估的功能。该功能入口在Dataset对象的stress_test方法中。

- 本文提供了一个使用stress_test进行压测的示例，通过本文，您可以快速上手我们的压测工具。

- 由于Locust中使用了gevent库来保证高并发性能，而gevent的高并发依赖于monkey patching的非阻塞I/O机制，但该机制在Notebook环境中默认未开启。因此，在开始测试前，需要进行monkey patching操作。这样做是为了确保整个环境，包括IPython/Jupyter自己的组件，都使用gevent兼容的非阻塞版本，从而避免因混合使用阻塞和非阻塞操作导致的不一致性和潜在的死锁问题。

In [1]:
from gevent import monkey
monkey.patch_all()

开始压测：

In [2]:
import os

# 开启压测服务
os.environ['QIANFAN_ENABLE_STRESS_TEST'] = "true"

from qianfan.dataset import Dataset

# 请在此处填写您的ak、sk
os.environ["QIANFAN_ACCESS_KEY"] = "..."
os.environ["QIANFAN_SECRET_KEY"] = "..."

# 需要初始化一个数据集对象
ds = Dataset.load(data_file="/data_file/stress_test.jsonl")

ds.multi_stress_test(
    origin_users=2,
    workers=2,
    spawn_rate=2,
    model= "ERNIE-Speed-8K",
    #endpoint="YourEndpoint",
    model_type="ChatCompletion",
    runtime="10s",
    rounds=3,
    interval=2,
    first_latency_threshold = 2,
    round_latency_threshold = 30,
    success_rate_threshold = 0.8,
)

[WARNING][2024-06-27 15:19:50.157] chunk_reader.py:36 [t:4407381824]: ijson aren't installed, only online function can be used
[INFO][2024-06-27 15:19:50.209] dataset.py:408 [t:4407381824]: no data source was provided, construct
[INFO][2024-06-27 15:19:50.222] dataset.py:276 [t:4407381824]: construct a file data source from path: /Users/jianruitian/Downloads/bce-qianfan-sdk-main/cookbook/stress/st.jsonl, with args: {}
[INFO][2024-06-27 15:19:50.238] file.py:293 [t:4407381824]: use format type FormatType.Jsonl
[INFO][2024-06-27 15:19:50.267] utils.py:349 [t:4407381824]: start to get memory_map from /Users/jianruitian/.qianfan_cache/dataset/Users/jianruitian/Downloads/bce-qianfan-sdk-main/cookbook/stress/st.arrow
[INFO][2024-06-27 15:19:50.289] utils.py:277 [t:4407381824]: has got a memory-mapped table
Type     Name                                                                          # reqs      # fails |    Avg     Min     Max    Med |   req/s  failures/s
--------|------------------

各项指标含义如下：

- **user_num**: 压测使用的本轮user数，即本轮发压的并发数；
- **worker_num**: 压测使用的worker数目，即进程数；
- **spawn_rate**: 每秒启动的user数目；
- **model_type**: 被压测服务的模型类型；
- **hyperparameters**: 压测使用的超参数；
- **QPS**：服务每秒实际处理的请求数；
- **Latency Avg/Min/Max/50%/80%**：全长时延的平均值/最小值/最大值/50分位值/80分位值；
- **FirstTokenLatency Avg/Min/Max/50%/80%**：首句时延的平均值/最小值/最大值/50分位值/80分位值；
- **InputTokens Avg**：单次请求输入的token长度平均值；
- **OutputTokens Avg**：单次请求输出的token长度平均值；
- **TotalQuery/SuccessQuery/FailureQuery**：总请求数/成功请求数/失败请求数；
- **TotalTime**：总运行时间；
- **SuccessRate**：请求成功率；

您也可以在当前路径下的record中查看相应的log文件。